In [11]:
from cdapython import Q,unique_terms,columns
import requests

In [25]:
bq_url="http://35.192.60.10:8080/api/v1/boolean-query/all_merged_subjects_v3_3_final"
bq_params={'dryRun':'false','table':'gcd-bq-sample.dev'}
postgres_url="http://localhost:8082/api/v1/boolean-query"
# postgres_url="https://cancerdata.ah-cda6.bee.envs-terra.bio/api/v1/boolean-query"
postgres_params={'dryRun':'false','includeCount':'false','offset':0,'limit':100}
headers={'accept':'application/json','Content-Type': 'application/json'}

In [61]:
myquery = Q('primary_diagnosis_site = "brain"')

In [65]:
bq_results = myquery.ORDER_BY("subject_id").run()

Getting results from database

Total execution time: 0
                            min 4.812 sec 4812 ms

In [53]:
print(myquery.to_dict())

{
    'node_type': '=',
    'l': {'node_type': 'column', 'value': 'primary_diagnosis_site'},
    'r': {'node_type': 'quoted', 'value': 'brain'}
}

In [66]:
bq_results._api_response

{'next_url': '/api/v1/query/36bcd6d0-d3fa-4d35-962c-becb44f1f735?offset=200&limit=100',
 'query_sql': 'SELECT results.* EXCEPT(rn) FROM (SELECT ROW_NUMBER() OVER '
              '(PARTITION BY Subject.id) as rn, Subject.id AS subject_id, '
              'Subject.identifier AS subject_identifier, Subject.species AS '
              'species, Subject.sex AS sex, Subject.race AS race, '
              'Subject.ethnicity AS ethnicity, Subject.days_to_birth AS '
              'days_to_birth, Subject.subject_associated_project AS '
              'subject_associated_project, Subject.vital_status AS '
              'vital_status, Subject.days_to_death AS days_to_death, '
              'Subject.cause_of_death AS cause_of_death FROM '
              'broad-dsde-prod.cda_prod.all_merged_subjects_v3_3_final AS '
              'Subject LEFT JOIN UNNEST(Subject.ResearchSubject) AS '
              '_ResearchSubject WHERE '
              "(IFNULL(UPPER(_ResearchSubject.primary_diagnosis_site), '') = "
  

In [40]:
def get_postgres_results(json_query):
    response = requests.post(postgres_url, params=postgres_params, headers=headers, json=json_query)
    print(response.request.url)
    print(response.request.body)
    print(response.json())
    print(response.status_code)


In [59]:
pg_results = get_postgres_results(myquery.to_dict())

http://localhost:8082/api/v1/boolean-query?dryRun=false&includeCount=false&offset=0&limit=100

b'{"node_type": "=", "l": {"node_type": "column", "value": "primary_diagnosis_site"}, "r": {"node_type": "quoted", 
"value": "brain"}}'

{
    'result': [
        {
            'subject_id': 'Broad.Burdenko_1360',
            'species': 'Homo sapiens',
            'sex': None,
            'race': None,
            'ethnicity': None,
            'days_to_birth': None,
            'vital_status': None,
            'days_to_death': None,
            'cause_of_death': None,
            'subject_associated_project': ['PDC000430', 'PDC000431', 'PDC000432', 'PDC000433'],
            'subjet_identifier': [
                {'system': 'PDC', 'field_name': 'Case.case_id', 'value': 'e0865cab-fcae-41a2-b51c-b49614be7f11'},
                {'system': 'PDC', 'field_name': 'Case.case_submitter_id', 'value': 'Burdenko_1360'}
            ]
        },
        {
            'subject_id': 'Broad.MB018',
            'species': 'Homo sapiens',
            'sex': 'Female',
            'race': None,
            'ethnicity': None,
            'days_to_birth': None,
            'vital_status': None,
            'days_to_death': None,
            'cause_of_death': None,
            'subject_associated_project': ['PDC000430', 'PDC000431', 'PDC000432', 'PDC000433'],
            'subjet_identifier': [
                {'system': 'PDC', 'field_name': 'Case.case_id', 'value': '60a125d8-c9d9-4092-b9d5-c22182b5c0e4'},
                {'system': 'PDC', 'field_name': 'Case.case_submitter_id', 'value': 'MB018'}
            ]
        },
        {
            'subject_id': 'Broad.MB037',
            'species': 'Homo sapiens',
            'sex': 'Female',
            'race': None,
            'ethnicity': None,
            'days_to_birth': None,
            'vital_status': None,
            'days_to_death': None,
            'cause_of_death': None,
            'subject_associated_project': ['PDC000430', 'PDC000431', 'PDC000432', 'PDC000433'],
            'subjet_identifier': [
                {'system': 'PDC', 'field_name': 'Case.case_id', 'value': 'cb4522d8-9b62-47af-9e5b-52bc868edec4'},
                {'system': 'PDC', 'field_name': 'Case.case_submitter_id', 'value': 'MB037'}
            ]
        },
        {
            'subject_id': 'Broad.MB088',
            'species': 'Homo sapiens',
            'sex': 'Male',
            'race': None,
            'ethnicity': None,
            'days_to_birth': None,
            'vital_status': None,
            'days_to_death': None,
            'cause_of_death': None,
            'subject_associated_project': ['PDC000430', 'PDC000431', 'PDC000432', 'PDC000433'],
            'subjet_identifier': [
                {'system': 'PDC', 'field_name': 'Case.case_id', 'value': 'f024723c-8b33-4505-841a-a6ff9ea4cada'},
                {'system': 'PDC', 'field_name': 'Case.case_submitter_id', 'value': 'MB088'}
            ]
        },
        {
            'subject_id': 'Broad.MB091',
            'species': 'Homo sapiens',
            'sex': 'Male',
            'race': None,
            'ethnicity': None,
            'days_to_birth': None,
            'vital_status': None,
            'days_to_death': None,
            'cause_of_death': None,
            'subject_associated_project': ['PDC000430', 'PDC000431', 'PDC000432', 'PDC000433'],
            'subjet_identifier': [
                {'system': 'PDC', 'field_name': 'Case.case_id', 'value': 'f5719333-bb76-4355-b59b-a35510350492'},
                {'system': 'PDC', 'field_name': 'Case.case_submitter_id', 'value': 'MB091'}
            ]
        },
        {
            'subject_id': 'Broad.MB095',
            'species': 'Homo sapiens',
            'sex': 'Male',
            'race': None,
            'ethnicity': None,
            'days_to_birth': None,
            'vital_status': None,
            'days_to_death': None,
            'cause_of_death': None,
            'subject_associated_project': ['PDC000430', 'PDC000431', 'PDC000432', 'PDC000433'],
            'subjet_identifier': [
                {'system': 'PDC', 'field_name': 'Case.case_id', 'value': '7b3b844f-47ea-4765-aa3e-9ec2a41fb510'},
                {'system'

200

In [52]:
pg_results

NameError: name 'JSON' is not defined